In [1]:
import argparse
import math
import os

import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter

from helper import pmath
from helper.helper import get_optimizer, load_dataset 
from helper.hyperbolicLoss import PeBusePenalty
from models.cifar import resnet as resnet_cifar
from models.cifar import densenet as densenet_cifar
from models.cub import resnet as resnet_cub
# from models.syndat import fullcon as fullcon_syndat

In [3]:
def main_train(model, trainloader, optimizer, initialized_loss, c=1.0):
    # Set mode to training.
    model.train()
    avgloss, avglosscount, newloss, acc, newacc = 0., 0., 0., 0., 0.

    # Go over all batches.
    for bidx, (data, target) in enumerate(trainloader):
        # bidx: Number of batches -> reaches from 0 - len(trainloader)
        # dara: btach_size tensors
        # Target: A tensor containing numbers from 0-100 (sice its cifar 100, I assume these
        # are the numbers of the class - basically the target and the y in my code)

        # Data to device.
        target_tmp = target # .cuda()

        target = model.polars[target]
        data = torch.autograd.Variable(data) # .cuda()
        target = torch.autograd.Variable(target) # .cuda()
        # Compute outputs and losses.
        output = model(data)
        output_exp_map = pmath.expmap0(output, c=c)

        loss_function = initialized_loss(output_exp_map, target)

        # Backpropagation.
        optimizer.zero_grad()
        loss_function.backward()
        optimizer.step()

        avgloss += loss_function.item()
        avglosscount += 1.
        newloss = avgloss / avglosscount

        output = model.predict(output_exp_map).float()
        pred = output.max(1, keepdim=True)[1]
        acc += pred.eq(target_tmp.view_as(pred)).sum().item()

    trainlen = len(trainloader.dataset)
    newacc = acc / float(trainlen)

    # I am returning new loss to show in the tensorboard!
    return newacc, newloss


def main_test(model, testloader, initialized_loss, c=1.0):
    # Set model to evaluation and initialize accuracy and cosine similarity.
    model.eval()
    acc = 0
    loss = 0

    # Go over all batches.
    with torch.no_grad():
        for data, target in testloader:
            # Data to device.
            data = torch.autograd.Variable(data) # .cuda()
            target = target.cuda(non_blocking=True)
            target = torch.autograd.Variable(target)
            target_loss = model.polars[target]

            # Forward.
            output = model(data).float()
            output_exp_map = pmath.expmap0(output, c=c)

            output = model.predict(output_exp_map).float()
            pred = output.max(1, keepdim=True)[1]
            acc += pred.eq(target.view_as(pred)).sum().item()

            loss += initialized_loss(output_exp_map, target_loss) # .cuda())

    # Print results.
    testlen = len(testloader.dataset)

    avg_acc = acc / float(testlen)
    avg_loss = loss / float(testlen)

    return avg_acc, avg_loss

In [4]:
logdir = '/Users/dj/Desktop'
data_name = 'cifar100'
do_decay = True
curvature = 1
kwargs = {'num_workers': 32, 'pin_memory': True}
datadir = 'data/'
resdir = 'runs/output_dir/cifar/' 
hpnfile = 'prototypes/prototypes-50d-100c.npy' 
logdir =  'test'
optimizer = 'adam'
learning_rate = 0.0005
momentum= 0.00005
decay = True
network = 'resnet32'
penalty = 0.1
mult = 0.1

# I want to use tensorboard to check the loss changes
log_dir = os.path.join('./runs/' + data_name, logdir)
writer = SummaryWriter(log_dir=log_dir)

# Set the random seeds.
seed = 100
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

In [5]:
# Load data.
batch_size = 10
trainloader, testloader = load_dataset(dataset_name = data_name,
        basedir = datadir,
            batch_size = batch_size,
                kwargs = kwargs) 

Files already downloaded and verified


/opt/anaconda3/envs/MasterThesis/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [6]:
if not os.path.exists(resdir):
    os.makedirs(resdir)

# Load the polars and update the trainy labels.
classpolars = torch.from_numpy(np.load(hpnfile)).float()
# calculate radius of ball
# This part is useful when curvature is not 1.
radius = 1.0 / math.sqrt(curvature)
classpolars = classpolars * radius

# hpnfile name is like prototypes-xd-yc.npy : x : dimension of prototype, y: number of classes
output_dims = int(hpnfile.split("/")[-1].split("-")[1][:-1])
print('This is the output_dims:', output_dims)

This is the output_dims: 50


In [7]:
# Load the model.
if (data_name == "cifar100") or (data_name == "cifar10"):
    if network == "resnet32":
        model = resnet_cifar.ResNet(32, output_dims, 1, classpolars)
    elif network == "densenet121":
        model = densenet_cifar.DenseNet121(output_dims, classpolars)
    else:
        print('The model you have chosen is not available. I am choosing resnet for you.')
        model = resnet_cifar.ResNet(32, output_dims, 1, classpolars)
elif args.data_name == "cub":
    if args.network == "resnet32":
        model = resnet_cub.ResNet34(args.output_dims, classpolars)
    else:
        print('The model you have chosen is not available. I am choosing resnet for you.')
        model = resnet_cub.ResNet34(args.output_dims, classpolars)
elif args.data_name == "syndat":
    if args.network == "fullcon":
        print('The fully connected Network has been chosen for the Synthetic Data.')
        model = fullcon_syndat.fullcon(dims = args.dims, output_dims = args.output_dims, dr = args.dr, polars = classpolars)
        print('Model has been activated.')
    elif args.network == "fullcon_selu":
        print('The fully connected SELU Network has been chosen for the Synthetic Data.')
        model = fullcon_syndat_selu.fullcon(dims = args.dims, output_dims = args.output_dims, dr = args.dr, polars = classpolars)
        print('Model has been activated.')
    elif args.network == "fullcon_lrelu":
        print('The fully connected Leaky RELU Network has been chosen for the Synthetic Data.')
        model = fullcon_syndat_lrelu.fullcon(dims = args.dims, output_dims = args.output_dims, dr = args.dr, polars = classpolars)
        print('Model has been activated.')
else:
    raise Exception('Selected dataset is not available.')

# model = model.to(device)
print('First time model initialization.')

# Load the optimizer.
optimizer = get_optimizer(optimizer, model.parameters(), learning_rate, momentum, decay)

# Initialize the loss functions.
choose_penalty = penalty
f_loss = PeBusePenalty(output_dims, penalty_option=choose_penalty, mult=mult)

First time model initialization.
~~~~~~~~!Your option is not available, I am choosing!~~~~~~~~


In [8]:
# Main loop.
epochs = 3
drop1 = 1
drop2 = 2
do_decay = True
testscores = []
learning_rate = learning_rate
for i in range(epochs):
    print(i)

    # Learning rate decay.
    if i in [drop1, drop2] and do_decay:
        learning_rate *= 0.1
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate

    # Train and test.
    acc, loss = main_train(model, trainloader, optimizer, f_loss, c=curvature)

    # add the train loss to the tensorboard writer
    writer.add_scalar("Loss/train", loss, i)
    writer.add_scalar("Accuracy/train", acc, i)

    if i != 0 and (i % 10 == 0 or i == epochs - 1): # i % 10
        test_acc, test_loss = main_test(model, testloader, f_loss, c=curvature)

        testscores.append([i, test_acc])

        writer.add_scalar("Loss/test", test_loss, i)
        writer.add_scalar("Accuracy/test", test_acc, i)

writer.flush()
writer.close()

0


RuntimeError: Boolean alpha only supported for Boolean results.